In [7]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.config import config


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import getpass

# Si no tenemos la configuración, usar esto

#API_KEY = getpass.getpass("Enter your API key: ")

API_KEY = config["OPENAI"]["API_KEY"]

In [9]:
from openai import OpenAI

client = OpenAI(
    api_key=API_KEY
)

In [10]:
from datasets import load_dataset

dataset = load_dataset("piuba-bigdata/contextualized_hate_speech")

In [11]:
dataset["dev"]

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 9106
})

In [12]:
from rioplatense_hs.preprocessing import labels, label_to_text

df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()
df_dev = dataset["dev"].to_pandas()
df_train["num_hatred"] = df_train[labels].sum(axis=1)
df_dev["num_hatred"] = df_dev[labels].sum(axis=1)
df_test["num_hatred"] = df_test[labels].sum(axis=1)

df_train["text_label"] = df_train.apply(lambda x: label_to_text(x), axis=1)
df_dev["text_label"] = df_dev.apply(lambda x: label_to_text(x), axis=1)
df_test["text_label"] = df_test.apply(lambda x: label_to_text(x), axis=1)


In [13]:
labels

['WOMEN',
 'LGBTI',
 'RACISM',
 'CLASS',
 'POLITICS',
 'DISABLED',
 'APPEARANCE',
 'CRIMINAL']

In [14]:
import pandas as pd

pd.options.display.max_colwidth = 1000
df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

,title,text,text_label
8021,"Quién es Ritchie Torres, el primer afrolatino homosexual elegido para el Congreso de EEUU","@usuario Negro, puto y zurdo. Esos son los méritos del tipo este. Es un congresista puesto por el establishment.","lgbti, racismo, política"
27278,Coronavirus en la Argentina: fue a donar plasma tras recuperarse pero se lo impidieron por ser gay,"@usuario Si sos gay no podés donar, si declarás que sos promiscuo no podés donar,si usaste drogas no podés donar, si te tatuaste hace menos de 6 meses no podés donar. ¿De quién es la culpa, del que dice la verdad al momento de desear hacerlo, o de los protocolos retrógrados que tenemos?",lgbti
22575,"Ricky Martin: ""Soy un hombre latino y homosexual viviendo en los Estados Unidos, soy una amenaza""","@usuario @usuario el problema no es que seas latino y trolo, el problema es que en el peor de los casos sos un lobbista hijo de mil puta y en el mejor de los casos sos un imbecil mononeuronal, o a ver si te atreves a declarar lo mismo en paises con ley sharia, cobarde sin verguenza","lgbti, racismo, discapacidad"
21617,"Pibita trans, activista negra, ni hombre ni mujer: ""Mi cuerpo es mi resistencia""",@usuario Pibita trans=nene,lgbti
22514,"Ricky Martin: ""Soy un hombre latino y homosexual viviendo en los Estados Unidos, soy una amenaza""","@usuario @usuario Proba con ir a triunfar a un país árabe, a ver qué tal te va, MARICON",lgbti
22559,"Pablo Duggan, sobre su casamiento: ""Karen es la mujer de mi vida, estoy muy enamorado""","@usuario Ah, también panquequea el toor...","mujer, lgbti"
8648,Luis Novaresio le dedicó un romántico mensaje a Braulio Bauab por su cumpleaños,@usuario quien será la pasiva?,lgbti
31588,"El llanto de Gustavo López tras recuperarse del coronavirus: ""La incertidumbre de no saber si salís es muy fuerte""",@usuario NO SEAS TROLO MAN,lgbti
28223,"Las lágrimas de Guillermo Andino tras recuperarse de coronavirus: ""El miedo es no poder contarla""","@usuario No seas trolo, man.",lgbti
27933,"Tras los comentarios de J.K. Rowling, Emma Watson defendió al colectivo trans",@usuario No veo fallas en la logica de JK para que salte el come trava de 22 pesos de Harry y la sucia inmunda. Va a salir el colorado a decir q se garcho un travesti y le gusto,"lgbti, apariencia"


In [15]:
# Agregar Chain of thought
# Agregar que es español rioplatense
template_prompt = f"""Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

mujer: Mujeres
lgbti: Género, identidad sexual
racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
clase: Clase, situación socioeconómica o barrio de residencia
discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol, drogas u otros estupefacientes
política: Opinión o ideología política
aspecto: Aspecto o edad
criminal: Antecedentes penales o estar privado de la libertad

En contexto tendremos un titular de una noticia, y en texto el comentario a analizar. Responder una o varias de las características separadas por coma, o bien "nada" si no hay discurso discriminatorio. Pensá paso a paso la respuesta.
##
contexto: Al borde del llanto, Nati Jota pidió disculpas por sus repudiables tuits pero los justificó: 'Quería sumar seguidores'
texto: Show o nada mi amorrr!!
La imbesilidad al palo!
salida: Es un insulto y no menciona nada relacionado a las características protegidas. La respuesta final es "nada"
##
contexto: Violento motín en la cárcel de Devoto: tomaron e incendiaron pabellones
texto: Hermanito lo que son esos grones. Ahí adentro le robaron la billetera al coronavirus
salida: La palabra "grones" significa negros, lo cual alude a su color de piel y aspecto. Además, se refiere a su condición de criminales. La respuesta final es "criminal, racismo, aspecto"
##
contexto: Mayra Mendoza criticó el banderazo contra el Gobierno: "No tiene sentido y es antidemocrático"
texto: Que negrita villera....ni con la plata q roba puede arreglarse
salida: El texto alude a su color de piel, a su condición social, y a su aspecto. La respuesta final es "mujer, racismo, aspecto, clase"
##
contexto: No da tregua: sin ropa y desafiante Flor Peña volvió a mostrar su lado más hot
texto: K lado hot tiene más varices k mi abuela. Tiene mas puesta de espalda k karagian trola
salida: El texto alude a su aspecto y la desprecia por 'trola'. La respuesta final es "mujer, aspecto"
##
contexto: Wuhan celebra el fin de la cuarentena por el coronavirus con un mensaje para el resto del mundo: "Aprendan de nuestros errores"
texto: Chinos culiados de mierda ojalá se mueran todos
salida: Desea que se mueran los chinos por ser los supuestos culpables del COVID-19. La respuesta final es "racismo"
##
contexto: Otro ataque de furia contra un colectivero: una mujer trans le pateó la unidad y le tiró un ladrillazo
texto: @usuario Un tipo operado. Con la fuerza de un hombre y no la de una mujer
salida: El texto alude a que la mujer trans es un hombre. La respuesta final es "lgbti"
##
"""



In [18]:
# sample_df = df_test.sample(100)

sample_df = df_dev.sample(700, random_state=2022)

In [19]:
(sample_df["num_hatred"] > 0).mean()

0.16857142857142857

In [20]:
outs = {

}

In [21]:
print(template_prompt)

Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

mujer: Mujeres
lgbti: Género, identidad sexual
racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
clase: Clase, situación socioeconómica o barrio de residencia
discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol, drogas u otros estupefacientes
política: Opinión o ideología política
aspecto: Aspecto o edad
criminal: Antecedentes penales o estar privado de la libertad

En contexto tendremos un titular de una noticia, y en texto el comentario a analizar. Responder una o varias de las características separadas por coma, o bien "n

In [23]:
from tqdm import tqdm
import time
from rioplatense_hs.preprocessing import preprocess_tweet
from pysentimiento.preprocessing import preprocess_tweet as _pysent_preprocess


def get_completion(prompt):
    return client.chat.completions.create(messages=[{
            "role": "user",
            "content": prompt
    }], model="gpt-3.5-turbo")

for idx, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):

    if idx in outs:
        continue

    contexto = _pysent_preprocess(row["context_tweet"], preprocess_hashtags=False, demoji=False, preprocess_handles=False)
    texto = preprocess_tweet(row["text"])
    prompt = template_prompt + f"contexto: {contexto}\ntexto: {texto}\nsalida: "

    response = get_completion(prompt)

    outs[idx] = response.choices[0].message.content

    time.sleep(0.025)

  0%|          | 0/700 [00:00<?, ?it/s]

100%|██████████| 700/700 [05:33<00:00,  2.10it/s] 


In [ ]:
import json

with open("outputs.json", "w") as f:
    json.dump(outs, f)


In [ ]:
labels = []

In [ ]:
from collections import Counter

counter = Counter()


for v in outs.values():
    for cl in v.split(","):
        counter[cl.strip().lower()] += 1


In [ ]:

counter

In [24]:
sample_df["pred_cot"] = sample_df.index.map(outs)


In [25]:
sample_df[["context_tweet", "text", "text_label","pred_cot"]]

,context_tweet,text,text_label,pred_cot
6934,Coronavirus: un diario inglés asegura que los mercados chinos siguen vendiendo murciélagos y carne de animales domésticos https://t.co/qFPs4aHf6D,@usuario Que gente de mierda .,nada,"El texto no contiene ningún tipo de referencia a características protegidas ni expresiones discriminatorias. La respuesta final es ""nada""."
5708,"Murió el periodista Mario Pereyra, ícono de la radio de Córdoba https://t.co/MZtxmqFP4o",@usuario Cuánto zurdito resentido en los comentarios...,política,"El texto utiliza el término ""zurdito"" de manera despectiva para referirse a personas de ideología política izquierdista. La respuesta final es ""política""."
7287,"""Se terminó, a salir a la calle”: detuvieron a un hombre que se grabó llamando a romper la cuarentena obligatoria https://t.co/VCK2uezc9p",@usuario Flor de imbécil,nada,"El texto contiene un insulto, pero no se dirige a ninguna característica protegida. La respuesta final es ""nada""."
5575,El polémico mensaje de Cinthia Fernández sobre la muerte de un delincuente en un intento de robo https://t.co/odv7PghHxL,@usuario Y que tiene tiene de polémico???,nada,"El texto no contiene discurso discriminatorio. La respuesta final es ""nada""."
6454,Coronavirus: China anunció que no registró ningún caso nuevo local https://t.co/bg8d1fSwBX https://t.co/M47osbSpVL,@usuario Chinos de mierda q se maten todos,racismo,"Desea que se maten los chinos por ser los supuestos culpables del COVID-19. La respuesta final es ""racismo"""
...,...,...,...,...
3204,"El contundente mensaje de Noelia Marzol sobre el feminismo y el sexo: ""Queremos coger en libertad"" https://t.co/Quo3h68jdU",@usuario Si.lo.hacen,nada,"No se menciona ninguna característica protegida en el texto. La respuesta final es ""nada""."
102,Alberto Fernández: “La pandemia finalmente nos dejó más muertos de los que creíamos tener” https://t.co/0T7hXsGS65,"@usuario baja el dedito , goberna y deja de gritarnos",nada,"El texto contiene una crítica hacia Alberto Fernández, pero no se basa en ninguna de las características protegidas mencionadas. La respuesta final es ""nada""."
2061,"Tras las críticas del kirchnerismo duro, Alberto Fernández llamará a Maduro para explicarle la condena argentina en la ONU | Por Román Lejtman https://t.co/yYnNvu50oO","@usuario ""Entendeme, Nicolás. Vos sos un poco brusco con los ciudadanos. Si lo taparas un poco tal vez, pero así de obvio no se puede""",nada,"El texto no contiene discurso discriminatorio, solamente expresa una opinión sobre la forma de actuar de Nicolás Maduro. La respuesta final es ""nada""."
8457,Los mapuches avanzaron en la toma de tierras en la Patagonia: cada hectárea está valuada en USD 100 mil https://t.co/52gdoBivK6,"@usuario Que los caguen a balazos y los echen como perros.\nUn tipo que labura todo el día y aporta al Estado no puede adquirir un triste terreno, y si con suerte lo adquiere, tiene que solicitar un crédito o préstamo para poder construir.",racismo,"El texto desea la violencia y discriminación hacia los mapuches por su condición de pertenecer a un pueblo aborigen. Además, se refiere a su condición de ""perros"" y alude a su situación socioeconómica. La respuesta final es ""racismo, clase"


In [26]:
sample_df.to_csv("../data/sample_dev_df.csv", index=False)

In [27]:
## Error analysis

In [28]:
import pandas as pd

pd.options.display.max_colwidth = 1000

sample_df = pd.read_csv("../data/sample_dev_df.csv")

In [29]:
sample_df[["context_tweet", "text", "text_label", "pred_cot"]]

,context_tweet,text,text_label,pred_cot
0,Coronavirus: un diario inglés asegura que los mercados chinos siguen vendiendo murciélagos y carne de animales domésticos https://t.co/qFPs4aHf6D,@usuario Que gente de mierda .,nada,"El texto no contiene ningún tipo de referencia a características protegidas ni expresiones discriminatorias. La respuesta final es ""nada""."
1,"Murió el periodista Mario Pereyra, ícono de la radio de Córdoba https://t.co/MZtxmqFP4o",@usuario Cuánto zurdito resentido en los comentarios...,política,"El texto utiliza el término ""zurdito"" de manera despectiva para referirse a personas de ideología política izquierdista. La respuesta final es ""política""."
2,"""Se terminó, a salir a la calle”: detuvieron a un hombre que se grabó llamando a romper la cuarentena obligatoria https://t.co/VCK2uezc9p",@usuario Flor de imbécil,nada,"El texto contiene un insulto, pero no se dirige a ninguna característica protegida. La respuesta final es ""nada""."
3,El polémico mensaje de Cinthia Fernández sobre la muerte de un delincuente en un intento de robo https://t.co/odv7PghHxL,@usuario Y que tiene tiene de polémico???,nada,"El texto no contiene discurso discriminatorio. La respuesta final es ""nada""."
4,Coronavirus: China anunció que no registró ningún caso nuevo local https://t.co/bg8d1fSwBX https://t.co/M47osbSpVL,@usuario Chinos de mierda q se maten todos,racismo,"Desea que se maten los chinos por ser los supuestos culpables del COVID-19. La respuesta final es ""racismo"""
...,...,...,...,...
695,"El contundente mensaje de Noelia Marzol sobre el feminismo y el sexo: ""Queremos coger en libertad"" https://t.co/Quo3h68jdU",@usuario Si.lo.hacen,nada,"No se menciona ninguna característica protegida en el texto. La respuesta final es ""nada""."
696,Alberto Fernández: “La pandemia finalmente nos dejó más muertos de los que creíamos tener” https://t.co/0T7hXsGS65,"@usuario baja el dedito , goberna y deja de gritarnos",nada,"El texto contiene una crítica hacia Alberto Fernández, pero no se basa en ninguna de las características protegidas mencionadas. La respuesta final es ""nada""."
697,"Tras las críticas del kirchnerismo duro, Alberto Fernández llamará a Maduro para explicarle la condena argentina en la ONU | Por Román Lejtman https://t.co/yYnNvu50oO","@usuario ""Entendeme, Nicolás. Vos sos un poco brusco con los ciudadanos. Si lo taparas un poco tal vez, pero así de obvio no se puede""",nada,"El texto no contiene discurso discriminatorio, solamente expresa una opinión sobre la forma de actuar de Nicolás Maduro. La respuesta final es ""nada""."
698,Los mapuches avanzaron en la toma de tierras en la Patagonia: cada hectárea está valuada en USD 100 mil https://t.co/52gdoBivK6,"@usuario Que los caguen a balazos y los echen como perros.\nUn tipo que labura todo el día y aporta al Estado no puede adquirir un triste terreno, y si con suerte lo adquiere, tiene que solicitar un crédito o préstamo para poder construir.",racismo,"El texto desea la violencia y discriminación hacia los mapuches por su condición de pertenecer a un pueblo aborigen. Además, se refiere a su condición de ""perros"" y alude a su situación socioeconómica. La respuesta final es ""racismo, clase"


In [22]:
# Check "la respuesta final" is in every pred_cot

sample_df[~sample_df["pred_cot"].str.contains("la respuesta final")]

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL,num_hatred,text_label,pred,pred_cot
0,405167,Limpiar la basura arrojada en la marcha por Milagro Sala costó 570 mil pesos,@usuario Aquien le envian la factura ????,Limpiar la basura arrojada en la marcha por Milagro Sala costó 570 mil pesos https://t.co/q9utmSZXKq,0,"Unas cinco mil pesonas protestaron el sábado, convocadas por La Campora y otras agrupaciones kirchneristas, contra la Corte Suprema y por la liberación de Milagro Sala, condenada por el desvío de unos $60 millones para viviendas que no se construyeron y que desaparecieron.\n\nLas imágenes mostraban decenas de bolsas de basura arrojadas en el frente del Palacio de Tribunales. Las cinco mil personas que no guardaron la distancia social se movilizaron y organizaron una puesta en escena a la que denominaron ""El basurero de la democracia"", en alusión de la Corte.\n\nAsí fue que arrojaron unas cuatro toneladas de residuos que, una vez terminada la protesta, fue recogida por el Gobierno de la Ciudad.\n\n¿Cuánto le costó a la Ciudad la limpieza y el arreglo del césped de la Plaza Lavalle? La módica suma de $572 mil que incluyó el hidrolavado y el barrido manual. A eso habría que sumarle el costo por seguridad, es decir, los agentes que fueron sacados de fuerzas especiales para disponerlas ...",0,0,0,0,0,0,0,0,0,0,nada,nada,"El texto no contiene ninguna expresión discriminatoria. La respuesta final es ""nada"""
1,375861,"Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer: así se inspiró para su segundo film","@usuario Yo no digo que este mal, o que no pase, pero no lo expongan como si fuese un modelo a seguir","Contó que era lesbiana, su papá le confesó que era gay y ahora su madre se enamoró de una mujer, lo que inspiró su segundo film | Por Hugo Martin\nhttps://t.co/dt3NtrVJIW",0,"Eldorado, Misiones. ""A treinta brazadas de nado, cruzando el río Paraná, está Paraguay. A 90 kilómetros, la Triple Frontera"", calcula la cineasta, guionista y Majo Staffolani. Habla del lugar donde nació el 13 de junio de 1990, de una familia humilde, un patio de tierra colorada, una calle con el empedrado desparejo y el plato de comida compartido entre los vecinos, por si no alcanzaba a todos. Del escenario, en definitiva, donde la misionera de 30 años disfrazaba sus deseos. ""Los mentía, porque desde los cuatro años se que soy lesbiana"", confiesa quien, por estos días, espera como a un mesías que llegue el 8 de octubre. Será esa la fecha del estreno de Román, su segundo film, que por la pandemia será emitido a través de la plataforma de Cine.ar. Se la escucha tan ansiosa como el torrente de palabras que llegan por el celular.\n\nLa película (que protagonizan Carlo Argento y Gastón Cocchiarale y cuenta con Vanessa Ragone -El secreto de sus ojos- como productora asociada) relata la ...",0,0,0,0,0,0,0,0,0,0,nada,ninguna,"El texto no contiene discurso discriminatorio. No se hacen declaraciones de rechazo, enemistad o aborrecimiento hacia un individuo o grupo por poseer una característica protegida. La respuesta final es ""nada""."
2,353018,El ex presidente uruguayo José Mujica renunció al Senado y se retira de la política activa,@usuario se va a cortar pasto con la uña del pie. !!,El ex presidente uruguayo José Mujica renunció al Senado y se retira de la política activa https://t.co/ubQZbsZlhh,0,"El ex presidente uruguayo José Mujica (2010-2015) renunció este martes a su escaño en el Senado y se retiró así definitivamente de la política activa.\n\nLa pandemia de coronavirus precipitó la decisión del ex mandatario de 85 años, que padece una enfermedad inmune. ""Esta situación me obliga, con mucho pesar por mi honda vocación política, a solicitar que gestione mi renuncia a la banca que me otorgó la ciudadanía"", escribió Mujica en una carta leída este martes en sesión extraordinaria del Senado.\n\n""Esto no significa el abandono de la polít

In [31]:
from rioplatense_hs.preprocessing import text_to_label, labels


pred_labels = [f"PRED_{l}" for l in labels]

for idx, value in sample_df["pred_cot"].items():

    preds = text_to_label(value)

    for k, v in preds.items():
        sample_df.loc[idx, f"PRED_{k}"] = int(v)

In [32]:
from sklearn.metrics import classification_report
gold_labels = sample_df[labels]
preds = sample_df[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.33      0.43      0.38        21
       LGBTI       0.40      0.62      0.48        13
      RACISM       0.55      0.82      0.66        40
       CLASS       0.15      0.40      0.22        10
    POLITICS       0.19      0.59      0.29        22
    DISABLED       0.33      0.20      0.25         5
  APPEARANCE       0.00      0.00      0.00        22
    CRIMINAL       0.24      0.35      0.29        17

   micro avg       0.32      0.49      0.39       150
   macro avg       0.28      0.43      0.32       150
weighted avg       0.30      0.49      0.37       150
 samples avg       0.09      0.09      0.09       150



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
 